<a href="https://colab.research.google.com/github/AKIYAMA-Keito/Colab-repo/blob/main/expcurve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import math
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
datanum = 1000

In [3]:
x_train = []
y_train = []
train_dataset = []
for i in range(datanum):
  x = -2
  while x < -1 or 1 < x:
    x = np.random.randn()
  print(x)
  # x = x[..., tf.newaxis].astype("float32")
  y = np.exp(-(x ** 2) / 2) / np.sqrt(2 * math.pi)
  x_train.append(x)
  y_train.append(y)
  train_dataset.append((x, y))

0.05919337240446362
0.7422820770827353
-0.21316556758819658
0.9217337918391509
-0.7403369484430299
-0.6796023842441418
0.6318454072838926
0.6565340394433584
-0.7147244819428054
-0.5572389604412357
0.63171009280221
0.41376351466518513
0.1634067234903248
0.7811532154927452
0.7937143234548675
-0.02803822391468806
0.06715930967498768
-0.0005909825520111416
0.9203022342824224
0.16064408239166986
0.6653700875520685
-0.5349437983321707
-0.1782558929590635
0.6300089098661753
0.6162388051198286
-0.4170835000039121
-0.0171326064899106
0.3151697197757042
-0.6850129072823631
-0.40404498714383963
-0.12445265274705089
-0.6807445581727265
0.026460548513163474
-0.7763610615317297
-0.3129992048216776
0.16271608958497696
0.2477123894225252
0.4169495356779637
-0.1821346785985963
-0.8503297551516905
0.3706701990301223
0.047040930521896014
-0.787276604761723
0.6936437588528043
0.03935596515730849
0.8810823045081986
0.6531913061646888
0.7813507403563227
-0.15634238490600014
0.2837083017495132
-0.84924544845

In [4]:
x_test = []
y_test = []
test_dataset = []
for i in range(datanum):
  x = -2
  while x < -1 or 1 < x:
    x = np.random.randn()
  # x = x[..., tf.newaxis].astype("float32")
  y = np.exp(-(x ** 2) / 2) / np.sqrt(2 * math.pi)
  x_test.append(x)
  y_test.append(y)
  test_dataset.append((x, y))

In [5]:
x_train = x_train[:datanum]
y_train = y_train[:datanum]
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
# x_train = x_train / 255.0
# x_test = x_test/255.0
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [6]:
batch_size = 100

In [7]:
train_tfds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)
).batch(batch_size)
test_tfds = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)
).batch(batch_size)

In [8]:
# print(train_tfds)

In [9]:
class CNNModel(Model):
    def __init__(self):
        super(CNNModel, self).__init__()
        # self.conv1 = Conv2D(32, 3, activation = "relu")
        # self.flatten = Flatten()
        self.d1 = Dense(128, activation = "relu")
        self.d2 = Dense(1)
    
    def call(self, x):
        # x = self.conv1(x)
        # x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        return x

model = CNNModel()

In [10]:
# model.summary()

In [11]:
loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam()

In [12]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.MeanAbsoluteError(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.MeanAbsoluteError(name='test_accuracy')

In [13]:
@tf.function
def train_step(inputs, outputs):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training = True)
        # print(predictions)
        loss = loss_object(outputs, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(outputs, predictions)

In [14]:
@tf.function
def test_step(inputs, outputs):
    predictions = model(inputs, training = False)
    t_loss = loss_object(outputs, predictions)
    
    test_loss(t_loss)
    test_accuracy(outputs, predictions)

In [15]:
# print(x_train[0])

In [16]:
# model(x_train[0], training = False)

In [17]:
EPOCHS = 100

for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    for train_inputs, train_outputs in train_tfds:
      # print((train_inputs.shape, train_outputs.shape))
      train_step(train_inputs, train_outputs)
    
    for test_inputs, test_outputs in test_tfds:
        test_step(test_inputs, test_outputs)
    
    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result()}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result()}'
    )

    if test_loss.result() * datanum < 1/8:
      break

Epoch 1, Loss: 0.11020712554454803, Accuracy: 0.31850963830947876, Test Loss: 0.074791319668293, Test Accuracy: 0.255109965801239
Epoch 2, Loss: 0.05111529678106308, Accuracy: 0.20478729903697968, Test Loss: 0.03488553315401077, Test Accuracy: 0.17193178832530975
Epoch 3, Loss: 0.02501300908625126, Accuracy: 0.1409425586462021, Test Loss: 0.01770961284637451, Test Accuracy: 0.11686579883098602
Epoch 4, Loss: 0.013842681422829628, Accuracy: 0.10163263976573944, Test Loss: 0.009752671234309673, Test Accuracy: 0.08667724579572678
Epoch 5, Loss: 0.008344708010554314, Accuracy: 0.08015473186969757, Test Loss: 0.006441630423069, Test Accuracy: 0.07032591104507446
Epoch 6, Loss: 0.005836444906890392, Accuracy: 0.06663323938846588, Test Loss: 0.00479153310880065, Test Accuracy: 0.05913401395082474
Epoch 7, Loss: 0.004162929020822048, Accuracy: 0.05561224743723869, Test Loss: 0.003253373084589839, Test Accuracy: 0.04846350476145744
Epoch 8, Loss: 0.0027303616516292095, Accuracy: 0.0449576005339

In [18]:
weights = model.get_weights()
for weight in weights:
  v = np.random.normal(0, 128 ** (-1.73))
  weight += v
model.set_weights(weights)

In [19]:
test_loss.reset_states()
test_accuracy.reset_states()

for test_inputs, test_outputs in test_tfds:
    test_step(test_inputs, test_outputs)
    
print(
    # f'Epoch {epoch + 1}, '
    # f'Loss: {train_loss.result()}, '
    # f'Accuracy: {train_accuracy.result()}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result()}'
)

Test Loss: 0.0001211513445014134, Test Accuracy: 0.009477530606091022


In [20]:
norm = test_loss.result() * datanum

In [21]:
print(norm)

tf.Tensor(0.12115134, shape=(), dtype=float32)


In [22]:
0.00011765809176722541 - 0.00011765795352403075

1.382431946694851e-10